In [1]:
import numpy as np
import os
from sklearn.decomposition import PCA
import glob

In [2]:
script_dir=os.getcwd()
rel_path_i='Input_data/DREBIN/feature_vectors/'
abs_path_i=os.path.join(script_dir,rel_path_i)
rel_path_d='Input_data/DREBIN/'
abs_path_d=os.path.join(script_dir,rel_path_d)
rel_path_m='Models/'
abs_path_m=os.path.join(script_dir,rel_path_m)

In [3]:
malicious_files=np.genfromtxt(abs_path_d+'sha256_family.csv',dtype=str,delimiter=',',skip_header=1)

In [4]:
np.shape(malicious_files)

(5560, 2)

In [5]:
no_of_mal=5560
labels=np.zeros((129013))

In [6]:
file_list=glob.glob(abs_path_i+'/*')
# file_list_2=np.genfromtxt(abs_path_d+'DREBIN_file_list.txt',dtype='S')

In [ ]:
labels_2=np.zeros((129013))

In [7]:
mal_set=malicious_files[:,0]
feature_set=set()
eight_features=set()
all_data=[]
file_count=0
for file_path in file_list:
    words=file_path.split('/')
    if words[-1] in mal_set:
        labels[file_count]=1
#         print words[-1], file_count
    curr_array=np.genfromtxt(file_path,dtype=str,delimiter="   ",usecols=(0))
    if np.ndim(curr_array)==0:
        curr_array=curr_array.reshape(1)
    for item in curr_array:
        feature_set.add(item)
        parts=item.split('::')
        eight_features.add(parts[0])
    all_data.append(curr_array)
    file_count=file_count+1

/usr/local/lib/python2.7/dist-packages/numpy/lib/npyio.py:1487: UserWarning: genfromtxt: Empty input file: "/home/arjun/Dropbox/Research/ResearchProjects/Princeton_Prateek/ML+Security/C_D_M_mlsec/Input_data/DREBIN/feature_vectors/f6239ba0487ffcf4d09255dba781440d2600d3c509e66018e6a5724912df34a9"
  warnings.warn('genfromtxt: Empty input file: "%s"' % fname)
/usr/local/lib/python2.7/dist-packages/numpy/lib/npyio.py:1487: UserWarning: genfromtxt: Empty input file: "/home/arjun/Dropbox/Research/ResearchProjects/Princeton_Prateek/ML+Security/C_D_M_mlsec/Input_data/DREBIN/feature_vectors/b8949d6f9e90c404a6ca747973e86a94d5cef97ed4dca8890ea538ffe8870daa"
  warnings.warn('genfromtxt: Empty input file: "%s"' % fname)
/usr/local/lib/python2.7/dist-packages/numpy/lib/npyio.py:1487: UserWarning: genfromtxt: Empty input file: "/home/arjun/Dropbox/Research/ResearchProjects/Princeton_Prateek/ML+Security/C_D_M_mlsec/Input_data/DREBIN/feature_vectors/3de513a148400b457dd8d8fa9238804db3ec031a0b526d4a04b77e

In [ ]:
labels[1]

In [ ]:
eight_features

labels=np.genfromtxt('labels.txt')

file_list=np.genfromtxt('file_list.txt',dtype='S')

In [ ]:
labels[1]

In [ ]:
a=np.where(labels==1.0)

In [ ]:
a

In [ ]:
b=np.where(labels_2==1.0)

In [ ]:
b

In [ ]:
len(a[0])

In [ ]:
len(feature_set)

In [8]:
no_of_features=len(feature_set)

In [9]:
no_of_samples=len(all_data)

In [10]:
feature_list=list(feature_set)

In [ ]:
feature_list[0]

In [11]:
no_of_features

545333

In [12]:
from scipy.sparse import coo_matrix

sample_row_list=[]
feature_col_list=[]
sample_count=0
for sample in all_data:
     print sample_count
    dims=np.ndim(sample)
    curr_features=set(sample)
     elif dims<2:
         sample=sample.reshape((1,2))
         curr_features=set(sample[:,1])
    curr_f_list=[i for i,item in enumerate(feature_list) if item in curr_features]
    feature_col_list.extend(curr_f_list)
    curr_s_list=[sample_count for item in curr_f_list]
    sample_row_list.extend(curr_s_list)
     for i in range(no_of_features):
         if feature_list[i] in curr_features:
             sample_row_list.append(sample_count)
             feature_col_list.append(i)
    sample_count=sample_count+1
np.savetxt('col_list.txt',feature_col_list,fmt='%0u')
np.savetxt('row_list.txt',sample_row_list,fmt='%0u')

In [13]:
row=np.genfromtxt(abs_path_d+'DREBIN_row_list.txt')
col=np.genfromtxt(abs_path_d+'DREBIN_col_list.txt')

In [14]:
len(row)

6113087

In [15]:
len(col)

6113087

In [16]:
data=np.ones((len(col)))

In [17]:
data_matrix=coo_matrix((data,(row,col)),shape=(no_of_samples,no_of_features))

/usr/local/lib/python2.7/dist-packages/numpy/core/fromnumeric.py:2652: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  VisibleDeprecationWarning)


In [18]:
data_matrix.shape

(129013, 545333)

In [19]:
data_matrix_csc=data_matrix.tocsc()

In [20]:
data_matrix_train=data_matrix_csc[0:40000,:]

In [21]:
data_matrix_test=data_matrix_csc[40000:,:]

In [22]:
labels_train=labels[0:40000]

In [23]:
labels_test=labels[40000:]

In [24]:
from sklearn import svm

In [25]:
clf=svm.LinearSVC()
clf.fit(data_matrix_train,labels_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [26]:
clf.score(data_matrix_test,labels_test)

0.99195623111230946

In [27]:
mal_indices_test=np.where(labels_test==1.0)[0]
benign_indices_test=np.where(labels_test==0.0)[0]

In [30]:
mal_indices_test

array([    6,    12,    36, ..., 88963, 88968, 89000])

In [31]:
labels_test[mal_indices_test]

array([ 1.,  1.,  1., ...,  1.,  1.,  1.])

In [32]:
clf.score(data_matrix_test[mal_indices_test,:],labels_test[mal_indices_test])

0.88906455862977607

In [ ]:
np.where(clf.predict(data_matrix_test[mal_indices_test[0],:])!=1.0)

In [ ]:
clf.predict(data_matrix_test[46,:])

In [ ]:
counter=0.0
for item in mal_indices_test[0]:
    if clf.predict(data_matrix_test[item,:])[0]!=1.0:
        counter=counter+1
print counter

In [ ]:
clf.score(data_matrix_test[benign_indices_test,:],labels_test[benign_indices_test])

In [ ]:
len(np.where(labels_test==1.0)[0])

In [ ]:
len(np.where(labels_train==1.0)[0])

In [33]:
for i in range(1):
    print np.linalg.norm(clf.coef_[i,:])

13.8579637701


In [34]:
top_malware_features=np.argsort(clf.coef_[0,:])[::-1][:100]

In [35]:
top_malware_features

array([473112, 229850, 234374, 282304, 195002, 513051,  99672, 388316,
        28991,  25128, 102237, 139163, 105308, 544592, 297934, 358170,
       209403, 283882, 372185, 182144, 248694, 313046, 318827, 312226,
       438136, 221494, 498032, 327380, 149712, 382180, 119149, 127142,
       255230, 412369, 125653, 121556, 152294,   2603, 164518, 386209,
       291933, 290788, 363330, 464969, 215830, 490592, 329660, 438720,
       323170, 140478, 525386,  27588,  70879, 235942,  45623, 199094,
       111220, 347452, 432983, 355831,  38567, 341010, 287607,  62156,
       128861, 359242, 343089, 494059, 450037, 456939, 169140, 512443,
       412599, 316008, 143378, 335060, 160032, 401304, 534059, 329861,
       325441, 251529, 462245, 456324, 430105, 504081, 316152, 531471,
       105955,  22903, 399495, 327260, 194537, 100629, 103118, 171735,
       133253,   7649, 211067, 397173])

In [36]:
feature_array=np.array(feature_list)

In [37]:
w=clf.coef_[0]

In [38]:
feature_array[top_malware_features]

array(['service_receiver::.SoftKeyboardPRO', 'activity::.a_main',
       'feature::android.softwate.live_wallpaper',
       'activity::.FaceNiffGenActivity', 'activity::.MoviePlayer',
       'url::smsreplier.net', 'activity::.GalleryActivity',
       'activity::.SMSReplicatorSecret', 'activity::.ChristmasTimer',
       'activity::.FirstActivity',
       'service_receiver::com.android.md5.Settings', 'url::datatkr.info',
       'url::depot.bulks.jp', 'activity::.Mine',
       'activity::.MovieActivity', 'call::system/bin/su',
       'url::http://www.d1xz.net/sm/zhiwen.aspx?ITxt%3D',
       'service_receiver::.service.PlayerBindService',
       'service_receiver::com.apperhand.device.android.AndroidSDKProvider',
       'activity::.vEphGedQWhKRkQ', 'activity::.SMSBomber',
       'activity::.OperaMiniActivity', 'activity::.jGwHcWMJOAop',
       'url::http://datatkr.info/and53', 'url::http://datatkr.info/and88',
       'activity::.DownloadActivity', 'call::Cipher(DES)',
       'url::appserve

In [39]:
actual_features=np.where(w!=0)[0]

In [40]:
clf.decision_function(data_matrix_train[0,:])

array([-1.54386189])

In [41]:
clf.decision_function(data_matrix_train[1,:])

array([ 0.97619794])

In [42]:
all_data_array=np.array(all_data)

In [43]:
mal_test=all_data_array[mal_indices_test]

In [ ]:
clf.decision_function(data_matrix_test[12,:])

In [ ]:
mal_indices_test

In [45]:
allowed_features=set()
temp_list=['feature','intent','permission','provider','service_receiver','intent','activity']
for item in temp_list:
    allowed_features.add(item)

In [46]:
attack_features=[]
for item in actual_features:
    curr_feature=feature_list[item]
    parts=curr_feature.split('::')
    if parts[0] in allowed_features:
        attack_features.append(item)    

In [47]:
feature_array[attack_features]

array(['service_receiver::.widget.AbstractWidgetProvider$AutoSyncToogleService',
       'service_receiver::com.p1.chompsms.appwidget.WidgetEventReceiver',
       'service_receiver::com.p1.chompsms.security.SecurityService', ...,
       'activity::.SpareParts',
       'activity::.activity.NetCounterPreferences',
       'activity::.activities.HomeActivity'], 
      dtype='|S111')

In [48]:
min(w[attack_features])

-1.0093990812703413

In [49]:
np.sort(w[attack_features])[:100]

array([-1.00939908, -0.99606833, -0.93635254, -0.91920572, -0.8853095 ,
       -0.85214412, -0.74161013, -0.63032952, -0.61575093, -0.61346177,
       -0.58927446, -0.5839318 , -0.5839318 , -0.58230716, -0.5720107 ,
       -0.5720107 , -0.55593857, -0.54148786, -0.53287173, -0.51884647,
       -0.51865115, -0.50453323, -0.50453323, -0.49083106, -0.47670033,
       -0.46718101, -0.46718101, -0.46083346, -0.45705879, -0.45494148,
       -0.45494148, -0.45494148, -0.45494148, -0.41707788, -0.41352678,
       -0.40947161, -0.40011266, -0.40011266, -0.40011266, -0.4000766 ,
       -0.399393  , -0.39927213, -0.39781779, -0.3976834 , -0.39630407,
       -0.3870832 , -0.38457152, -0.38457152, -0.38313195, -0.38024377,
       -0.37711024, -0.37528755, -0.37273577, -0.37273577, -0.37273577,
       -0.37273577, -0.37189271, -0.36072691, -0.36072691, -0.36072381,
       -0.35890311, -0.35890311, -0.35890311, -0.35704786, -0.35704786,
       -0.35287343, -0.34624358, -0.34617122, -0.33567713, -0.33

### Choosing most negative features (most benign) to flip

In [50]:
w_index=np.argsort(w[attack_features])[:100]

In [51]:
attack_features_array=np.array(attack_features)

In [52]:
top_attack_f=attack_features_array[w_index]

In [53]:
top_attack_f

array([257065, 457352, 381442,  76803, 115600, 524339, 544747, 408629,
        89186, 374926, 172424, 187817, 236171, 103470, 333345, 333347,
        12942, 521102,  48679, 505200,  83282,   9096, 170457, 286094,
       327624, 344985, 526628, 235219,  81393, 539246, 436003, 331251,
       511146, 457499, 167000, 164870, 115209, 233193, 314456, 516788,
       537891, 392381, 415481, 135323, 439026,  56554, 126345,  98508,
        41006, 154184, 147245, 230383, 339743, 335037, 100214, 466971,
        47756, 434592, 251404, 134348,  90428, 368779, 520695, 490397,
       105559, 139088, 191271, 449670,   5140, 198327, 257604,  42720,
       295408,  38822, 410779, 368316, 134657,   9505, 110498,  50087,
       250401, 151262, 204198,  37395, 292234, 442533, 448080, 361093,
        55757, 358785, 161974, 245247,  50176, 210240, 149920, 420586,
       176617, 223321, 450274,   1129])

In [54]:
feature_array[top_attack_f[0:10]]

array(['service_receiver::collegepics.iconosys.eng.LocationSender',
       'service_receiver::realestatebuzzer.iconosys.eng.LocationSender',
       'feature::android.hardware.touchscreen',
       'service_receiver::com.apps.andromadhu.bmw1live.BootReceiver',
       'service_receiver::com.YRHNew.BootReceiver',
       'activity::.j2h3kj2h3jk2h3Activity', 'activity::.mainandroid1',
       'service_receiver::com.iapps.carraceslideme.BootReceiver',
       'activity::.HeroStarter', 'service_receiver::LocationSender'], 
      dtype='|S111')

In [55]:
mal_indices_test=np.array(mal_indices_test).reshape((1,-1))

In [56]:
mal_indices_test.shape

(1, 3795)

In [57]:
mal_indices_test.reshape((3795))

array([    6,    12,    36, ..., 88963, 88968, 89000])

In [59]:
mal_indices_test

array([[    6,    12,    36, ..., 88963, 88968, 89000]])

In [58]:
count_wrong=0.0
count_tot=0.0
count_adv=0.0
base_wrong=0.0
for index in mal_indices_test[0]:
    mal_test_sample=data_matrix_test[index,:]
    mal_test_sample[:,top_attack_f[0]]=1.0
    if clf.predict(data_matrix_test[index,:])[0]!=1.0:
        base_wrong=base_wrong+1
    if clf.predict(mal_test_sample)[0]!=1.0:
        count_wrong=count_wrong+1
    if clf.predict(mal_test_sample)[0]!=clf.predict(data_matrix_test[index,:])[0]:
        count_adv=count_adv+1
    count_tot=count_tot+1
print count_wrong/count_tot*100, count_adv/count_tot*100, base_wrong/count_tot

/usr/lib/python2.7/dist-packages/scipy/sparse/compressed.py:728: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


100.0 100.0 0.0


In [ ]:
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
svd=TruncatedSVD(n_components=5000)
svd.fit(data_matrix_train)

In [69]:
X_train_rd=svd.transform(data_matrix_train)
clf_svd=svm.LinearSVC()
clf_svd.fit(X_train_rd,labels_train)
X_test_rd=svd.transform(data_matrix_test)
clf_svd.score(X_test_rd,labels_test)

0.98773212901486296

In [70]:
mal_test_sample=data_matrix_test[mal_indices_test[0],:]
ini_score=clf.score(mal_test_sample,labels_test[mal_indices_test[0]])
mal_test_sample_thousand=svd.transform(mal_test_sample)
ini_score_thousand=clf_svd.score(mal_test_sample_thousand,labels_test[mal_indices_test[0]])
for i in range(10):
    mal_test_sample[:,top_attack_f[i]]=1.0
    curr_score=clf.score(mal_test_sample,labels_test[mal_indices_test[0]])
    mal_test_sample_thousand=svd.transform(mal_test_sample)
    curr_score_thousand=clf_svd.score(mal_test_sample_thousand,labels_test[mal_indices_test[0]])
    print i,curr_score,i,curr_score_thousand

0 0.494861660079 0 0.797891963109
1 0.125164690382 1 0.798682476943
2 0.124637681159 2 0.798682476943
3 0.0363636363636 3 0.799209486166
4 0.0129117259552 4 0.798682476943
5 0.00553359683794 5 0.799472990777
6 0.00237154150198 6 0.799472990777
7 0.0 7 0.799472990777
8 0.0 8 0.799736495389
9 0.0 9 0.823451910408


In [71]:
from sklearn.externals import joblib
joblib.dump(svd,"DREBIN_svd_500.pkl")

['DREBIN_svd_500.pkl',
 'DREBIN_svd_500.pkl_01.npy',
 'DREBIN_svd_500.pkl_02.npy',
 'DREBIN_svd_500.pkl_03.npy']

In [ ]:
mal_test_sample_thousand.shape

In [65]:
ini_score

0.88906455862977607

In [66]:
ini_score_thousand

0.66613965744400527

In [ ]:
mal_test_sample

In [ ]:
mal_test_sample[:,top_attack_f[0]]=1.0

In [ ]:
mal_test_sample[:,top_attack_f[1]]=1.0

In [ ]:
mal_test_sample[:,top_attack_f[2]]=1.0

In [ ]:
mal_test_sample[:,top_attack_f[3]]=1.0

In [ ]:
mal_test_sample

In [ ]:
clf.score(mal_test_sample,labels_test[mal_indices_test[0]])

In [ ]:
clf.predict(mal_test_sample[0,:])[0]

In [ ]:
clf.decision_function(mal_test_sample)

In [ ]:
clf.decision_function(data_matrix_test[,:])

In [ ]:
labels_test[0]

In [ ]:
X_train_rd=svd.transform(data_matrix_train)

In [ ]:
X_train_rd.shape

In [ ]:
clf_svd=svm.LinearSVC()
clf_svd.fit(X_train_rd,labels_train)

In [ ]:
clf_svd.score(X_train_rd,labels_train)

In [ ]:
clf_svd.predict(X_train_rd[0,:])

In [ ]:
X_test_rd=svd.transform(data_matrix_test)

In [ ]:
clf_svd.score(X_test_rd,labels_test)

In [ ]:
mal_rd=svd.transform(mal_test_sample)

In [ ]:
clf_svd.predict(mal_rd)

In [ ]:
clf.predict(mal_test_sample)

In [ ]:
count_wrong=0.0
count_tot=0.0
count_adv=0.0
base_wrong=0.0
for index in test_mal_indices:
    mal_test_sample=data_matrix_test[index,:]
    mal_test_sample[:,top_attack_f[0]]=1.0
    mal_rd=svd.transform(mal_test_sample)
    if clf_svd.predict(X_test_rd[index,:])[0]!=1.0:
        base_wrong=base_wrong+1
    if clf_svd.predict(mal_rd)[0]!=1.0:
        count_wrong=count_wrong+1
    if clf_svd.predict(mal_rd)[0]!=clf_svd.predict(X_test_rd[index,:])[0]:
        count_adv=count_adv+1
    count_tot=count_tot+1
print count_wrong/count_tot*100, count_adv/count_tot*100, base_wrong/count_tot

In [ ]:
from sklearn.externals import joblib

In [ ]:
joblib.dump(svd,"DREBIN_svd.pkl")
joblib.dump(clf_svd,"DREBIN_SVM_dr.pkl")
joblib.dump(clf_svd,"DREBIN_SVM.pkl")

In [ ]:
svd_2=joblib.load(abs_path_m+'DREBIN_svd.pkl')

In [ ]:
svd_2

In [ ]:
X_test_rd=svd_2.transform(data_matrix_test)

In [ ]:
X_train_rd=svd_2.transform(data_matrix_train)

In [ ]:
clf_svd=svm.LinearSVC()
clf_svd.fit(X_train_rd,labels_train)

In [ ]:
clf_svd.score(X_test_rd,labels_test)

In [ ]:
mal_rd=svd_2.transform(mal_test_sample)

In [ ]:
clf_svd.score(mal_rd,labels_test[mal_indices_test[0]])